In [1]:
import random
import binascii

MILLER_RABIN_TRIAL_NUM = 1000

def gcd(a, b):
    return a if b == 0 else gcd(b, a % b)

def egcd(a, b):
    if b == 0:
        return a, 1, 0
    else:
        d, x, y = egcd(b, a % b)
        return d, y, x - (a // b) * y
    
def modular_linear_equation_solver(a, b, n):
    ret = []
    d, x, y = egcd(a, n)
    if b % d == 0:
        x_0 = (x * (b / d)) % n
        for i in range(0, d):
            ret.append((x_0 + i * (n / d)) % n)
    return ret

def mul_inv(a, n):
    d, x, y = egcd(a, n)
    if d == 1:
        return x % n
    else:
        raise Exception('gcd(a, n) != 1')
        
def crt(list_a, list_n):
    a, p = 0, 1
    for n in list_n:
        p = p * n
    for i in range(len(list_a)):
        a_i, n_i = list_a[i], list_n[i]
        m_i = p / n_i
        a += (a_i * m_i * mul_inv(m_i, n_i))
    return a % p

def modular_exponentiation(a, b, n):
    c = 0
    d = 1
    b_binary = "{0:b}".format(b)
    for i in range(0, len(b_binary)):
        c = 2 * c
        d = (d * d) % n
        if b_binary[i] == "1":
            c = c + 1
            d = (d * a) % n
    return d

def witness(a, n):
    t, u = 0, n - 1
    while u % 2 == 0:
        u >>= 1
        t += 1
    assert((2 ** t) * u == n - 1)
    x = modular_exponentiation(a, u, n)
    for _ in range(t):
        x_prev = x
        x = (x_prev ** 2) % n
        if x == 1 and x_prev != 1 and x_prev != n - 1:
            return True
    if x != 1:
        return True
    return False

def miller_rabin(n, s):
    for _ in range(s):
        a = random.randrange(1, n - 1)
        if witness(a, n):
            return "composite"
    return "prime"

def generate_primes(integer_bits):
    primes = []
    while len(primes) < 2:
        n = random.getrandbits(integer_bits)
        if miller_rabin(n, MILLER_RABIN_TRIAL_NUM) == "prime":
            primes.append(n)
    return primes[0], primes[1]

def generate_keys(integer_bits):
    p, q = generate_primes(integer_bits)
    n = p * q
    phi_n = (p - 1) * (q - 1)
    e = random.randrange(1, phi_n, 2)
    while gcd(e, phi_n) != 1:
        e = random.randrange(1, phi_n, 2)
    d = mul_inv(e, phi_n)
    P, S = (e, n), (d, n)
    return P, S

def encrypt(P, plain_text):
    e, n = P
    plain_integer = int(binascii.hexlify(plain_text.encode()), 16)
    return modular_exponentiation(plain_integer, e, n)

def decrypt(S, cipher_text):
    d, n = S
    decrypted_integer = modular_exponentiation(cipher_text, d, n)
    return binascii.unhexlify(hex(decrypted_integer)[2:]).decode()

In [2]:
P, S = generate_keys(1024)

In [3]:
print("public key = ({0}, {1})".format(hex(P[0]), hex(P[1])))
print("private key = ({0}, {1})".format(hex(S[0]), hex(S[1])))

public key = (0x1a84f31b54d81cc873dd8eefe66921400812918784c3f18e84c400d185b3136e0e6e2c27b3170654fc2882d2345d054d347932e6560cee92cde0d3e0edf92f4e70fb47c5d57d6e351628dc1ac33b956b517fdaa93d2dc2f908bcdd69e87c7b8951c55042dc2f74b1312a92ed1634705818231af978ea78d5ba0c684d149fcd07e67138b9e37b2124f731aa4492b801d5afcb765e0cd10cdb1e03d37e10a197eb1111326a7b4d74301f85071e7a486b2ceff4de1c6cdda4600bc99b04a2f58e1baf3a709623d1595b4608a34c88c6f163fc5f1d63cf6765ad8f6d8c02f277cab079310e959c32055c8cbf1ffb2315d86a9f337a4f1627025f5e684880ca7f62bf, 0x225f5d96fcdad2451cd2c89469deac035b5c9fd7d9d16ac9f340a9091762b8305bff773b7fb088cb576867a99455f3e267a5d6b491b1f67a6afbcdbf70246d1f701acab9ee47681fb894791f6f5764089413a359409e0701354c79b1fbc0117c8a9b578c68f7dc16a4e9ddcf26f9c6b213b3e44dbd9b1abbebb725301d54a07f9dfdc46d68e83c6b32a7108ae2eb3123d77c8a6b120765e021e7d42659518d7d317f802b349d2cab06e2fd36aacb261560c4e79479d40457bb3e6d2e6a2af42e38cbd6dc2e51420c317cfabe3540fa2a68e2c5a6049f691d710888aa0d06768f9f0289ae31decc2888f8

In [4]:
M = "Hello World!"
cipher_text = encrypt(P, M)
print(hex(cipher_text))

0x19e597e54ca3fd600dd1c68f213fd1dcf676a97aa2f8bb2cd89b8dd57d31f50c0e8c370ad4f819289f03ee9052f4ddbbd6c9bd51d8ddff861242f5195f4fd398c8c8ba26d979aaab4ae0662f9bd1660d0bdf31a9a63245f36aeb4fc9683cdec394b3cb7ec427cbf4e5a3f258f19a5e1c65405f5b11de3dfcaa56354e7ffb7f823f3555b2c264c7c894e4e85d4aeef6af271a7465cb48053b88d059ee7d8cc472881b89935763e3d013510c4b5a3eb80dc355972d86bbc0abb926d29177fcc2bfa7c52a64d301faf6eabfd3f402c1314c1fec0c03aa4d21c8e149bc7f1bc4fc453ae45c03656e59732bd1b02cc7bbcb1898c4582098c774b4680c21f03b94aa58


In [5]:
plain_text = decrypt(S, cipher_text)
print(plain_text)

Hello World!
